# 0.4 Phase 0 Execution Validation

**Objective:** Validate that we can actually execute trades profitably before building ML models.

This notebook:
1. Screens upcoming earnings for tradeable candidates
2. Places small test orders (1 contract straddles)
3. Logs all execution details
4. Builds fill probability and slippage models

**Target:** 20-50 fills to build empirical execution model.

**Gate to proceed:** Do NOT build ML models until fill model is validated.

In [1]:
import nest_asyncio
nest_asyncio.apply()

import sys
sys.path.insert(0, '..')

from ib_insync import IB
import pandas as pd
from datetime import datetime, date, timedelta
from pathlib import Path
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Import our modules
from trading.earnings import (
    fetch_upcoming_earnings,
    screen_all_candidates,
    Phase0Executor,
    TradeLogger,
)

# Settings
IB_HOST = '127.0.0.1'
IB_PORT = 4002  # Paper trading
CLIENT_ID = 5

SPREAD_THRESHOLD = 15.0  # Max spread % to trade
MAX_CONTRACTS = 1  # Start with 1 contract per leg
LIMIT_AGGRESSION = 0.3  # How far above mid to place limit (0.3 = 30% of spread)

## 1. Connect to IBKR

In [2]:
ib = IB()
ib.connect(IB_HOST, IB_PORT, clientId=CLIENT_ID)
print(f'Connected: {ib.isConnected()}')

# Use live market data (requires subscription)
# Change to 3 for delayed if no subscription
ib.reqMarketDataType(1)
print('Using live market data')

2026-01-09 11:57:17,574 - INFO - Connecting to 127.0.0.1:4002 with clientId 5...
2026-01-09 11:57:17,578 - INFO - Connected
2026-01-09 11:57:17,581 - INFO - Logged on to server version 176
2026-01-09 11:57:17,624 - INFO - Warning 2104, reqId -1: Market data farm connection is OK:usopt
2026-01-09 11:57:17,625 - INFO - Warning 2104, reqId -1: Market data farm connection is OK:usfarm
2026-01-09 11:57:17,626 - INFO - Warning 2107, reqId -1: HMDS data farm connection is inactive but should be available upon demand.ushmds
2026-01-09 11:57:17,627 - INFO - Warning 2158, reqId -1: Sec-def data farm connection is OK:secdefil
2026-01-09 11:57:17,628 - INFO - API connection ready
2026-01-09 11:57:17,631 - INFO - position: Position(account='DUO353296', contract=Option(conId=656214500, symbol='AEHR', lastTradeDateOrContractMonth='20260116', strike=22.5, right='P', multiplier='100', currency='USD', localSymbol='AEHR  260116P00022500', tradingClass='AEHR'), position=5.0, avgCost=204.90874)
2026-01-09 

Connected: True
Using live market data


2026-01-09 11:57:20,459 - INFO - updatePortfolio: PortfolioItem(contract=Option(conId=784784233, symbol='NEOG', lastTradeDateOrContractMonth='20260116', strike=7.5, right='P', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='NEOG  260116P00007500', tradingClass='NEOG'), position=-5.0, marketPrice=0.02956475, marketValue=-14.78, averageCost=5.29296, unrealizedPNL=11.68, realizedPNL=0.0, account='DUO353296')


In [4]:
# Check account
print("Account Summary:")
for av in ib.accountValues():
    if av.currency == 'EUR' and av.tag in ['NetLiquidation', 'AvailableFunds']:
        print(f"  {av.tag}: ${float(av.value):,.2f}")

Account Summary:
  AvailableFunds: $986,322.40
  NetLiquidation: $1,005,261.86


## 2. Fetch Upcoming Earnings

In [5]:
# Get earnings for next 7 days
events = fetch_upcoming_earnings(days_ahead=7)
print(f"Found {len(events)} upcoming earnings events")

# Show first 20
events_df = pd.DataFrame([{
    'symbol': e.symbol,
    'date': e.earnings_date,
    'timing': e.timing,
} for e in events[:20]])
print(events_df.to_string(index=False))

2026-01-09 11:57:37,277 - INFO - Fetched 58 earnings events from Nasdaq API


Found 58 upcoming earnings events
symbol       date  timing
  ANIX 2026-01-09 unknown
  HURC 2026-01-09 unknown
  LEXX 2026-01-09 unknown
  LEDS 2026-01-09 unknown
  SIFY 2026-01-12     BMO
  BNED 2026-01-12 unknown
  VWAV 2026-01-12 unknown
  SOTK 2026-01-12 unknown
  EDUC 2026-01-12 unknown
  VMAR 2026-01-12 unknown
  BAOS 2026-01-12 unknown
   JPM 2026-01-13     BMO
    BK 2026-01-13     BMO
   DAL 2026-01-13     BMO
  CNXC 2026-01-13     BMO
  KARO 2026-01-13 unknown
  PXED 2026-01-13     AMC
   PKE 2026-01-13 unknown
  BBCP 2026-01-13     AMC
   PLG 2026-01-13 unknown


2026-01-09 12:00:20,440 - INFO - updatePortfolio: PortfolioItem(contract=Option(conId=656214432, symbol='AEHR', lastTradeDateOrContractMonth='20260116', strike=22.5, right='C', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='AEHR  260116C00022500', tradingClass='AEHR'), position=5.0, marketPrice=1.69448235, marketValue=847.24, averageCost=219.90876, unrealizedPNL=-252.3, realizedPNL=0.0, account='DUO353296')
2026-01-09 12:00:20,440 - INFO - updatePortfolio: PortfolioItem(contract=Option(conId=656214500, symbol='AEHR', lastTradeDateOrContractMonth='20260116', strike=22.5, right='P', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='AEHR  260116P00022500', tradingClass='AEHR'), position=5.0, marketPrice=2.56503655, marketValue=1282.52, averageCost=204.90874, unrealizedPNL=257.97, realizedPNL=0.0, account='DUO353296')
2026-01-09 12:03:20,463 - INFO - updatePortfolio: PortfolioItem(contract=Option(conId=656214432, symbol='AEHR', lastTradeDateOrCo

## 3. Screen for Tradeable Candidates

In [ ]:
# Screen candidates (this takes a while - ~3 sec per symbol)
# Limit to 30 to keep it reasonable
print(f"Screening up to 30 candidates with spread threshold {SPREAD_THRESHOLD}%...")
print("This will take ~2 minutes...")

passed, rejected = screen_all_candidates(
    ib,
    events,
    spread_threshold=SPREAD_THRESHOLD,
    max_candidates=30,
)

print(f"\nResults: {len(passed)} passed, {len(rejected)} rejected")

In [ ]:
# Show passed candidates
if passed:
    print("\n" + "="*60)
    print("TRADEABLE CANDIDATES")
    print("="*60)
    
    passed_df = pd.DataFrame([{
        'symbol': c.symbol,
        'earnings': c.earnings_date,
        'timing': c.timing,
        'spot': c.spot_price,
        'expiry': c.expiry,
        'strike': c.atm_strike,
        'straddle': c.straddle_mid,
        'spread%': c.spread_pct,
        'impl_move%': c.implied_move_pct,
    } for c in passed])
    
    print(passed_df.round(2).to_string(index=False))
else:
    print("No candidates passed liquidity screening")

In [ ]:
# Show rejection reasons
print("\n" + "="*60)
print("REJECTION REASONS")
print("="*60)

reasons = {}
for c in rejected:
    reason = c.rejection_reason or 'unknown'
    # Simplify reason for grouping
    if 'Spread too wide' in reason:
        reason = 'Spread too wide'
    reasons[reason] = reasons.get(reason, 0) + 1

for reason, count in sorted(reasons.items(), key=lambda x: -x[1]):
    print(f"  {reason}: {count}")

## 4. Initialize Executor and Logger

In [ ]:
# Initialize trade logger (SQLite database)
trade_logger = TradeLogger(db_path='../data/earnings_trades.db')

# Initialize executor
executor = Phase0Executor(
    ib=ib,
    trade_logger=trade_logger,
    max_contracts=MAX_CONTRACTS,
    limit_aggression=LIMIT_AGGRESSION,
)

print(f"Executor ready. Max contracts: {MAX_CONTRACTS}, Limit aggression: {LIMIT_AGGRESSION}")

## 5. Place Orders on Candidates

**WARNING:** This will place real orders in paper trading!

In [ ]:
# Log rejected candidates (for survivorship bias prevention)
for candidate in rejected:
    executor.log_non_trade(candidate)

print(f"Logged {len(rejected)} non-trades")

In [ ]:
# Select candidates to trade
# For Phase 0, start with just 1-2 to validate the flow
candidates_to_trade = passed[:2]  # Adjust as needed

if candidates_to_trade:
    print(f"Will place orders on {len(candidates_to_trade)} candidates:")
    for c in candidates_to_trade:
        print(f"  {c.symbol} - earnings {c.earnings_date} ({c.timing})")
else:
    print("No candidates to trade")

In [ ]:
# Place orders
# UNCOMMENT TO EXECUTE

# for candidate in candidates_to_trade:
#     print(f"\nPlacing order for {candidate.symbol}...")
#     order_pair = executor.place_straddle(candidate)
#     if order_pair:
#         print(f"  Order placed: {order_pair.trade_id}")
#     else:
#         print(f"  Order failed")

print("Orders commented out - uncomment to execute")

In [ ]:
# Check fill status
ib.sleep(5)  # Wait a bit for fills

filled = executor.check_fills()
print(f"\nFilled orders: {len(filled)}")

for order in filled:
    print(f"  {order.symbol}: Call @ ${order.call_fill_price:.2f}, Put @ ${order.put_fill_price:.2f}")

print(f"\nActive (unfilled) orders: {executor.get_active_count()}")

## 6. Monitor and Manage Orders

In [ ]:
# View all open orders
open_orders = ib.openTrades()
print(f"Open orders: {len(open_orders)}")

for trade in open_orders:
    c = trade.contract
    print(f"  {c.symbol} {c.right} {c.strike} - {trade.orderStatus.status}")

In [ ]:
# Cancel all orders if needed
# UNCOMMENT TO CANCEL

# executor.cancel_all()
# print("Cancelled all orders")

## 7. View Execution Logs

In [ ]:
# Get summary stats
stats = trade_logger.get_summary_stats()
print("Trade Log Summary:")
print(f"  Total trades: {stats['total_trades']}")
print(f"  Completed trades: {stats['completed_trades']}")
print(f"  Total P&L: ${stats['total_pnl']:.2f}")
print(f"  Total non-trades logged: {stats['total_non_trades']}")
print(f"\nRejection breakdown:")
for reason, count in stats['rejection_breakdown'].items():
    print(f"    {reason}: {count}")

In [ ]:
# View recent trades
trades = trade_logger.get_trades()
if trades:
    trades_df = pd.DataFrame([{
        'trade_id': t.trade_id[:20],
        'ticker': t.ticker,
        'status': t.status,
        'entry_mid': t.entry_quoted_mid,
        'fill_price': t.entry_fill_price,
        'slippage': t.entry_slippage,
    } for t in trades[:10]])
    print("Recent trades:")
    print(trades_df.to_string(index=False))
else:
    print("No trades yet")

In [ ]:
# View execution metrics (for Phase 0 analysis)
metrics = trade_logger.get_execution_metrics()
print("Execution Metrics:")
print(f"  Total orders: {metrics.total_orders}")
print(f"  Fill rate: {metrics.fill_rate*100:.1f}%")
print(f"  Avg slippage: {metrics.avg_slippage_bps:.1f} bps")
print(f"  Median slippage: {metrics.median_slippage_bps:.1f} bps")
print(f"  Max slippage: {metrics.max_slippage_bps:.1f} bps")

## 8. Cleanup

In [ ]:
ib.disconnect()
print("Disconnected from IB Gateway")

## Phase 0 Checklist

Run this notebook daily during earnings season to collect execution data.

**Target metrics (from V1 plan):**
- [ ] 30+ fills logged with full execution details
- [ ] Fill model: can predict fill probability within 10%
- [ ] Slippage model: realized within 1% of predicted
- [ ] Identified minimum viable liquidity thresholds

**Once complete:**
- Analyze fill rates by spread bucket
- Analyze slippage by OI bucket
- Set final liquidity thresholds based on empirical data
- Proceed to ML model development